In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import inv
from numpy.linalg import det
from numpy.linalg import eig
from numpy.linalg import cholesky
from scipy import optimize
import progressbar
import random

# Read data and initialize parameters

In [2]:
df = pd.read_excel("./macrometrics_codes/data_m_ready.xlsx",header=None)

In [3]:
# parameters 
nonlinear = "yes"
logistic = "yes"
interacted = "no"
statevar = "FFRshadow"
gamma = 10
cq = 13
lags = 6
h = 48
c_case = 2
var_list = ["GDPgap",'CoreCPIGr12','FFRshadow']
exog_vars = []
n = len(var_list)
ident = "chol"
shockpos = 3 #position of the shock in the cholesky ordering
shocksize = 1 #0=one standard deviation, all else: absolute values
nboot = 100 #bootstraps
alpha = 10 #90% confidence-interval 

In [4]:
# parameter dictionary 
params = {"non_linear":nonlinear
        ,"logistic":logistic
         ,"interacted":interacted
         ,"statevar":statevar
         ,"gamma":gamma
         ,"cq":cq
         ,"varlist":var_list
        ,"lags":lags
        ,"h":h
        ,"c_case":c_case
        ,"exog_vars":exog_vars
        ,"num_vars":n
        ,"identification":ident
        ,"shockposition":shockpos
        ,"shocksize":shocksize
        ,"nboot":nboot
        ,"CI":alpha}

In [5]:
# clean data
df = df.replace("NaN",np.NaN) \
       .rename(columns={10:"GDPgap",18:'FFRshadow',22:'CoreCPIGr12'}) \

time = pd.date_range('1967-01-01', periods = df.shape[0], freq='1M')
df["time"] = time
# select endogenous vars
y = df[params["varlist"]]

# select exogenous vars
exog = df[params["exog_vars"]].add_suffix("_exog")

# select state (if nescesarry)
if params["non_linear"] == "yes" and params["logistic"] == "yes":
    s = df[params["statevar"]].rename("state")

data = pd.concat([y,exog,s,df["time"]],axis=1)

# ignore NaN 
data = data.dropna().reset_index(drop=True)

# drop data before 2019
data = data[data.time.dt.year<2019].reset_index(drop=True)

# logistic transformation of state-variable
if params["non_linear"] == "yes" and params["logistic"] == "yes":
    data.state = (data.state - np.percentile(data.state,params["cq"],interpolation='midpoint')) / np.std(data.state)
    data["Fstate"] = np.exp(-params["gamma"]*data.state) / (1+np.exp(-params["gamma"]*data.state))
    
    # state dummy
    absval = np.percentile(data.state,params["cq"],interpolation='midpoint')
    data["state_dummy"] =[1 if s <= absval else 0 for s in data.state]

In [6]:
def prepare_data(df,params):
    """Prepare the data in accordance with specified"""
    # Prep
    s = df["state"]
    s = s.shift(1) # lagged one period 
    
    # y matrix
    y = df[params["varlist"]]
    
    # Generate lags
    lagged_vars = []
    
    for i in range(1,params["lags"]+1):
        lagged_vars.append(y.shift(i))
        
    # concat lagged vars and truncate    
    lagged_vars = pd.concat(lagged_vars,axis=1).dropna()
    
    # truncate other dataframes
    y = df[params["varlist"]][params["lags"]:]
    state = s[params["lags"]:].values.reshape(-1,1)
    T = len(y)
    
    # add constant and/or trend to X matrix
    if c_case == 0:
        c = np.NaN
    elif c_case == 1:
        c = np.ones(lagged_vars.shape[0])
    elif c_case == 2:
        list(zip(np.ones(lagged_vars.shape[0]),lagged_vars.index-lags))
    else:
        print('c_case variable needs to be set to 0, 1, or 2.')
        
    
    # add exogenous variables
    if params["exog_vars"]:
        exog = []
        for i in range(1,params["lags"]+1):
            Xex.append(params["exog_vars"].shift(i))
    else:
        exog = []
        print("No exogenous variables defined")
    
    return state,y,lagged_vars,T,exog

In [7]:
state,y,lagged_vars,T,exog = prepare_data(data,params)

No exogenous variables defined


In [8]:
def lag_criteria(omega,U,A):
    """ Returns the AIC and BIC lag selection criteria"""
    AIC = np.log(det(omega)) + 2/len(U)*len(A)
    BIC = np.log(det(omega)) + np.log(len(U))/len(U)*len(A)
    return AIC, BIC

In [9]:
def find_A_U_omega(lagged_vars,state,y,exog):
    ## Linear and nonlinear var for initial values
    rX = lagged_vars.shape[0]
    cX = lagged_vars.shape[1]
    et = np.ones((1,cX)) #vector of ones
    
    ## NONLINEAR
    
    # Check for exogenous variables
    if exog:
        Xm = np.block([lagged_vars, lagged_vars * np.kron(state,et), lagged_vars * np.kron(state**2*np.sign(state),et),np.ones((rX,1)),exog])
    else:
        Xm = np.block([lagged_vars, lagged_vars * np.kron(state,et), lagged_vars * np.kron(state**2*np.sign(state),et),np.ones((rX,1))])
        
    A = inv(np.dot(Xm.T,Xm))@(np.dot(Xm.T,y))
    U = y - Xm @ A
    omega = np.cov(U.T)
    
    # Lag selection criteria
    AIC, BIC = lag_criteria(omega,U,A)
    
    ## LINEAR BENCHMARK
    if exog:
        Xm_lin = np.block([lagged_vars,np.ones((rX,1)),exog])
    else:
        Xm_lin = np.block([lagged_vars,np.ones((rX,1))])
        
    A_lin = inv(np.dot(Xm_lin.T,Xm_lin))@(np.dot(Xm_lin.T,y))
    U_lin = y - Xm_lin @ A_lin
    omega_lin = np.cov(U_lin.T)
    A_lin = A_lin[:-1] #potentiel fejl
    
    return omega,U,A,omega_lin,U_lin,A_lin

In [10]:
omega,U,A,omega_lin,U_lin,A_lin = find_A_U_omega(lagged_vars,state,y,exog)

In [11]:
def find_lag_criteria(omega,U,A,omega_lin,U_lin,A_lin):
    """ Returns AIC and BIC lag selection criteria"""
    #non-linear model
    AIC, BIC = lag_criteria(omega,U,A)

    #linear benchmark modeel
    AIC_lin, BIC_lin = lag_criteria(omega_lin,U_lin,A_lin)
    print("--------")
    print("Lag selection criteria:")
    print("--------")
    print("Nonlinear model:")
    print(f"AIC = {AIC:.5f}")
    print(f"BIC = {BIC:.5f}")
    print("--------")
    print("Linear model:")
    print(f"AIC = {AIC_lin:.5f}")
    print(f"BIC = {BIC_lin:.5f}")
    print("--------")
    
    return

In [12]:
find_lag_criteria(omega,U,A,omega_lin,U_lin,A_lin)

--------
Lag selection criteria:
--------
Nonlinear model:
AIC = -7.37621
BIC = -6.97625
--------
Linear model:
AIC = -6.44517
BIC = -6.31428
--------


In [13]:
theta0 = -params["gamma"]
scale_penalty = 1e6
scale_penalty_beta = 0
lambda_prior = 1
theta_prior = 0.5

In [14]:
s_prior = {"theta0":theta0
          ,"scale_penalty":scale_penalty
          ,"scale_penalty_beta":scale_penalty_beta
          ,"lambda_prior":lambda_prior
          ,"theta_prior":theta_prior}

In [15]:
def minnesota_prior(params,omega_lin,s_prior):
    """
    shrinkage prior for autoregressive parameters in vector autoregressive (VAR) models
    
    Args:
    params(dict):
    num_vars = # of variables in the VAR
    lags = # of lags in the VAR
    
    s_prior(dict):
    - lambda_prior = controls cross-variable responses (smaller values mean stronger prior 
    - theta_prior = controls own response of variables
    
    Omega_lin = covariance matrix of residuals to make restrictions scale
    invariant (typically from "first stage" unrestricted VAR). (estimated in find_lag_criteria)
    
    Returns:
    matrix of tighness for restrictions
    """
    prior_mean = np.zeros((params["num_vars"],params["num_vars"]*params["lags"]))

    for i in range(params["num_vars"]):
        prior_mean[i,i]=1
    
    
    prior_var = np.zeros((params["num_vars"],params["num_vars"]*params["lags"]))

    for lag in range(params["lags"]):
        for i in range(params["num_vars"]):
            for j in range(params["num_vars"]):
                if i == j:
                    prior_var[i,j+lag*params["num_vars"]] = s_prior["lambda_prior"]**2 / ((lag+1)**2)
                else:
                    prior_var[i,j+lag*params["num_vars"]] = (s_prior["lambda_prior"]*s_prior["theta_prior"])**2 / ((lag+1)**2) * (omega_lin[i,i]/omega_lin[j,j])

    return prior_mean, prior_var

In [16]:
prior_mean, prior_var = minnesota_prior(params,omega_lin,s_prior)

In [17]:
s_prior["prior_mean"] = prior_mean
s_prior["prior_var"] = prior_var

In [18]:
def LSTVAR_mleScov(resids,state,omega_1,omega_2,theta,T,num_var):
    """ 
    this function computes the likelihood for the process the VAR with a smooth transtion (observed) state 
    Transition for state is given by two functions:
    
    slopes/intercept     = F(Z) = exp(gamma*Z)/(1+exp(gamma*Z))
    cov matrix of shocks = M(Z) = exp(theta*Z)/(1+exp(theta*Z))
    
    identifying assuumtpions are that gamma<0, theta<0
    
    compute VAR coefficients using GLS
    
    Args:
    resids = residuals from find_lag_criteria (u-vector)
    s = regime shifter for contemporanous response (matrices OMEGA)
    omega_1 = cov matrix of residuals in regime 0
    omega_2 = cov matrix of residuals in regime 1
    theta = transition speed for coevariance matrix of VAR residuals
    T = sample size
    """
    
    
    # weights on the "expansion" regime
    M_Z_t = np.exp(theta * state) / (1 + np.exp(theta * state))
    
    logL = 0 
    
    for i in range(0,T):
        #compute time-varying parameters
        omega_t = omega_1 * (1 - M_Z_t[i]) + omega_2 * M_Z_t[i]
        
        e_t = resids.values[i]
        
        logL = logL - 0.5*(np.log(det(omega_t)) + e_t @ inv(omega_t) @ e_t.T)

    return logL

In [19]:
def dupmat(n):
    """Returns Magnus and Neudecker's duplication matrix of size n"""
    
    a = np.tril(np.ones(n))
    i = np.where(a>0)
    a[i] = range(0,len(i[0]))
    a = a + np.tril(a,-1).T
    
    # create vector
    j = a.flatten("F")
    
    m = n*(n+1)/2;
    d = np.zeros((n*n,int(m)));
    
    for r in range(d.shape[0]):
        d[r,int(j[r])] = 1
    
    return d

In [20]:
dup_mat_n = dupmat(params["num_vars"])
d_plus = inv(np.dot(dup_mat_n.T,dup_mat_n))@dup_mat_n.T
omegalin_var = 2*d_plus@np.kron(omega_lin,omega_lin)@d_plus.T / T
s_prior["omega_var"] = s_prior["scale_penalty"]*np.ones(np.diag(omegalin_var).shape[0])
s_prior["omega_lin"] = omega_lin

In [21]:
#take a small departure for one of the regimes to make sure that starting
#values are slightly different
D=np.random.randn(len(omega),len(omega))
#omega_1 = omega
#omega_2 = omega + min(eig(omega)[0]) * D @ D.T
omega_1 = np.array([[0.1522,1.2358e-04,0.0122],[1.2358e-04,0.0311,0.0037],[0.0122,0.0037,0.1115]])
omega_2 = np.array([[0.1637,-0.0345,0.0431],[-0.0345,0.3561,-0.0826],[0.0431,-0.0826,0.2093]])

In [22]:
#Apply Cholesky decomposition since we need to look only for a lower
#trinagular matrix

In [23]:
s1 = cholesky(omega_1)
s2 = cholesky(omega_2)
s2 = s2.flatten("F")
omega_length_cov = len(s2[s2!=0])
s_desc = {"omega_length":omega_length_cov,"T":T}

In [24]:
param0 = np.block([s2[s2!=0],s2[s2!=0],s_prior["theta0"]])

In [25]:
def unvech(vec):
    """ creates a matrix from a vectorc """
    rows = .5 * (-1 + np.sqrt(1 + 8 * len(vec)))
    rows = int(np.round(rows))

    result = np.zeros((rows, rows))
    counter1 = 0 
    for row in range(rows):
        for i in range(row,rows):
            result[i,row] = vec[counter1]
            counter1 +=1

    for row in range(rows):
        for i in range(row,rows):
            result[row,i] = result[i,row]

    return result

In [26]:
def vec2matScov(X):
    """ convert vector of parameters into approapriate matrices 
    
    Args:
    param0: Vector of parameters
    """
    
    # the curvature in the transition function
    theta = X[-1]
    
    #  covariance matrix of the error terms
    ## pick vectorized matrices from the vector of parameters, param0
    omega_1 = X[:s_desc["omega_length"]]
    omega_2 = X[s_desc["omega_length"]:-1]
    
    # unvectorize matrices
    omega_1_m = unvech(omega_1)
    omega_2_m = unvech(omega_2)
    
    # Remove elements above diagonal
    omega_1_m = omega_1_m - np.triu(omega_1_m,1)
    omega_2_m = omega_2_m - np.triu(omega_2_m,1)
    
    # Ensure the covariance matrices are positive definite (this step is effectively using the nature of the Cholesky decomposition)
    omega_1 = omega_1_m @ omega_1_m.T
    omega_2 = omega_2_m @ omega_2_m.T
    
    return omega_1, omega_2,theta

In [27]:
def max_LSTVAR_mleScov(X,resids,state,s_desc,params,s_prior):
   
    omega_1, omega_2, theta = vec2matScov(X)
    
    if theta<0:
        if np.min(eig(omega_1)[0])>0 and np.min(eig(omega_2)[0])>0:
            obj_logl = LSTVAR_mleScov(resids,state,omega_1,omega_2,theta,s_desc["T"],params["num_vars"])
            
            #take the negative of the log-likelihood (because the objective
            #function is minimization rather than maximization) and add
            #penalty for deviation of theta (the curvature in the transition
            #probability) from the prior.  % take the negative of the log-likelihood (because the objective
            #function is minimization rather than maximization) and add
            #penalty for deviation of theta (the curvature in the transition
            #probability) from the prior. 
            
            obj_logl = -obj_logl + (theta-s_prior["theta0"])**2 * s_prior["scale_penalty"]
        else:
            obj_logl = 1e12
    else:
        obj_logl = 1e12

    return obj_logl

In [28]:
resids = U

In [29]:
res = optimize.minimize(max_LSTVAR_mleScov,param0,args=(resids,state,s_desc,params,s_prior), method = 'Nelder-Mead',options={'maxiter':1000,'maxfev':1000,'xatol':1e-6,'fatol':1e-6})

In [30]:
# unpack results
omega_1, omega_2, theta = vec2matScov(param0)
omega_1_opt, omega_2_opt, gamma_opt = vec2matScov(res.x)

In [31]:
print(f"log-likelihood: {res.fun}")
print("------")
print("Values before optimization")
print(f"omega_1: {omega_1.tolist()}")
print(f"omega_2: {omega_2.tolist()}")
print(f"theta: {theta}")
print("------")
print("Values after optimization")
print(f"omega_1: {omega_1_opt.tolist()}")
print(f"omega_2: {omega_2_opt.tolist()}")
print(f"theta: {gamma_opt}")

log-likelihood: -1311.8577672497593
------
Values before optimization
omega_1: [[0.16369999999999998, -0.0345, 0.0431], [-0.0345, 0.3561000000000001, -0.0826], [0.0431, -0.0826, 0.2093]]
omega_2: [[0.16369999999999998, -0.0345, 0.0431], [-0.0345, 0.3561000000000001, -0.0826], [0.0431, -0.0826, 0.2093]]
theta: -10.0
------
Values after optimization
omega_1: [[0.1444419801180465, -0.009410119804626173, 0.01782977023357277], [-0.009410119804626173, 0.043501824856164543, -0.036286624916261626], [0.01782977023357277, -0.036286624916261626, 0.19913780590079985]]
omega_2: [[0.5370169307598047, -0.08558955205462175, 0.10457335845479447], [-0.08558955205462175, 0.03412744757238577, -0.04288415803279071], [0.10457335845479447, -0.04288415803279071, 0.1016452172713204]]
theta: -9.999647599595


In [32]:
# cholesky
s1 = cholesky(omega_1_opt)
s1 = s1.flatten("F")
s2 = cholesky(omega_2_opt)
s2 = s1.flatten("F")

In [33]:
param0_MCMC =  np.block([s1[s1!=0],s2[s2!=0],gamma])

# Markov Chain Monte Carlo algorithm

In [34]:
rX = lagged_vars.shape[0]
cX = lagged_vars.shape[1]

In [35]:
def unvec(A,n1,n2):
    """ worker function to create matrix from vector 
    
    Args:
    A: input matrix
    n1: dimension 1
    n2: dimension 2
    
    Returns:
    v: matrix of dimension n1xn2
    """
    
    v = np.empty((n1,n2))
    for i in range(0,n2):
        v[:,i] = A[i*n1:(i+1)*n1]
    return v

In [36]:
def LSTVAR_struct(theta,gamma,state,lagged_vars,y,exog,omega_1,omega_2,T,c_case,lags,nvar,n_periods):
    """ 
    this function computes the likelihood for the process the VAR with a smooth transtion (observed) state 
    Transition for state is given by two functions:
    
    slopes/intercept     = F(Z) = exp(gamma*Z)/(1+exp(gamma*Z))
    cov matrix of shocks = M(Z) = exp(theta*Z)/(1+exp(theta*Z))
    
    identifying assuumtpions are that gamma<0, theta<0
    
    compute VAR coefficients using GLS
    
    Args:
    theta: transition speed for coevariance matrix of VAR residuals
    gamma: transition speed for VAR coefs
    s: regime shifter for contemporanous response (matrices OMEGA)
    lagged_vars: vector of lagged variables
    y: vector of dependent variables
    exog: vector of exogenous variables
    omega_1: cov matrix of residuals in regime 0
    omega_2: cov matrix of residuals in regime 1
    T: sample size
    c_case: controls trend, regime-specific intercept and regime-specific trend.
    lags: no. lags
    nvar: number of variables in the VAR
    n_periods = number of forecasted periods
    """
    if c_case == 0:
        trendON = 0 #0 = trend, 1 = no trend
        inter_regspec = 0 #1=regime-specific intercept
        trend_regspec = 0 #1= regime-specific trend
    elif c_case == 1:
        trendON = 0
        inter_regspec = 1
        trend_regspec = 0;
    elif c_case == 2:
        trendON = 1
        inter_regspec = 1
        trend_regspec = 1
    
    # weights on the "expansion" regime
    M_Z_t = np.exp(theta * state) / (1 + np.exp(theta * state))
    F_Z_t = np.exp(gamma * state) / (1 + np.exp(gamma * state))
    
    # construct vector of regressors
    # the last two entries capture constant terms and trends
    rX = lagged_vars.shape[0]
    cX = lagged_vars.shape[1]
    
    if trendON == 0:
        if inter_regspec == 0: 
            if parameters["lags"] > 0:
                if exog:
                    Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), np.ones((len(F_Z_t),1)),exog])
                else:
                    Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), np.ones((len(F_Z_t),1))])
            else: 
                Xm = np.ones((len(F_Z_t),1))
        else:
            if exog:
                Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), (1-F_Z_t), F_Z_t, exog])
            else:
                Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), (1-F_Z_t), F_Z_t])
    
    else: 
        if inter_regspec == 0 and trend_regspec == 0:
            if exog:
                Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), np.ones((len(F_Z_t),1)),np.array(range(len(F_Z_t))).reshape(-1,1), exog])
            else:
                Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), np.ones((len(F_Z_t),1)),np.array(range(len(F_Z_t))).reshape(-1,1)])
                
        elif inter_regspec == 1 and trend_regspec == 0: 
            if exog:
                Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), (1-F_Z_t), F_Z_t, np.array(range(len(F_Z_t))).reshape(-1,1), exog])
            else:
                Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), (1-F_Z_t), F_Z_t, np.array(range(len(F_Z_t))).reshape(-1,1)])
        
        elif inter_regspec == 0 and trend_regspec == 1: 
            if exog:
                Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), np.ones((len(F_Z_t),1)),(1-F_Z_t) * np.array(range(len(F_Z_t))).reshape(-1,1), F_Z_t * np.array(range(len(F_Z_t))).reshape(-1,1), exog])
            else:
                Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), np.ones((len(F_Z_t),1)),(1-F_Z_t) * np.array(range(len(F_Z_t))).reshape(-1,1), F_Z_t * np.array(range(len(F_Z_t))).reshape(-1,1)])
        
        elif inter_regspec == 1 and trend_regspec == 1: 
            if exog:
                Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), (1-F_Z_t), F_Z_t, (1-F_Z_t) * np.array(range(len(F_Z_t))).reshape(-1,1), F_Z_t * np.array(range(len(F_Z_t))).reshape(-1,1), exog])
            else:
                Xm = np.block([lagged_vars.values * np.tile(1 - F_Z_t,(1,cX)), lagged_vars.values * np.tile(F_Z_t,(1,cX)), (1-F_Z_t), F_Z_t, (1-F_Z_t) * np.array(range(len(F_Z_t))).reshape(-1,1), F_Z_t * np.array(range(len(F_Z_t))).reshape(-1,1)])
    
    # Estimate VAR coefficients using GLS
    num0 = 0
    den0 = 0
    
    for t in range(0,T):
        wgt_m = inv(omega_1 * (1 - M_Z_t[t]) + omega_2 * M_Z_t[t])
        num0_mat = Xm[t:].T @ y[t:].values @ wgt_m
        
        num0 = num0 + num0_mat.flatten("F")
        den0 = den0 + np.kron(wgt_m, Xm[t:].T @ Xm[t:])
        
    betaK = inv(den0) @ num0
    betaK1 = unvec(betaK,cX*2+trendON*(1+trend_regspec)+1+inter_regspec+n_periods, nvar)
    
    # VAR coefficients in regime 0
    beta0 = betaK1[:cX,:].T
    # VAR coefficients in regime 1
    beta1 = betaK1[cX:2*cX,:].T
    
    # compute coefficients on the constant term and any other control variables
    beta_const = betaK1[2*cX:2*cX+1,:].T
    beta_other = betaK1[2*cX+2:,:]
    
    # compute residuals
    residsM = y.values - Xm @ betaK1
    
    # compute log-likelihood
    logL = 0
    
    for i in range(T):
        # compute time-varying parameters
        omega_t = omega_1 * (1 - M_Z_t[i]) + omega_2 * M_Z_t[i]
        
        e_t = residsM[i]
        
        logL = logL - 0.5*(np.log(det(omega_t)) + e_t@inv(omega_t)@e_t.T)
    
    return logL, beta0, beta1, beta_const, beta_other

In [37]:
def vec2matSF_struct(X,state,lagged_vars,y,exog,T,c_case,lags,nvar,n_periods):
    """ 
    this function computes the likelihood for the process the VAR with a smooth transtion (observed) state 
    Transition for state is given by two functions:
    
    slopes/intercept     = F(Z) = exp(gamma*Z)/(1+exp(gamma*Z))
    cov matrix of shocks = M(Z) = exp(theta*Z)/(1+exp(theta*Z))
    
    identifying assuumtpions are that gamma<0, theta<0
    
    compute VAR coefficients using GLS
    
    Args:
    theta: transition speed for coevariance matrix of VAR residuals
    gamma: transition speed for VAR coefs
    s: regime shifter for contemporanous response (matrices OMEGA)
    lagged_vars: vector of lagged variables
    y: vector of dependent variables
    exog: vector of exogenous variables
    omega_1: cov matrix of residuals in regime 0
    omega_2: cov matrix of residuals in regime 1
    T: sample size
    c_case: controls trend, regime-specific intercept and regime-specific trend.
    lags: no. lags
    nvar: number of variables in the VAR
    n_periods = number of forecasted periods
    """
    omega_1, omega_2, theta = vec2matScov(X)
    gamma = theta
    
    if c_case == 0:
        trendON = 0 #0 = trend, 1 = no trend
        inter_regspec = 0 #1=regime-specific intercept
        trend_regspec = 0 #1= regime-specific trend
    elif c_case == 1:
        trendON = 0
        inter_regspec = 1
        trend_regspec = 0;
    elif c_case == 2:
        trendON = 1
        inter_regspec = 1
        trend_regspec = 1

    # weights on the "expansion" regime
    M_Z_t = np.exp(theta * state) / (1 + np.exp(theta * state))
    F_Z_t = np.exp(gamma * state) / (1 + np.exp(gamma * state))
    
    # construct vector of regressors
    # the last two entries capture constant terms and trends
    rX = lagged_vars.shape[0]
    cX = lagged_vars.shape[1]
    
    if trendON == 0:
        if inter_regspec == 0: 
            if parameters["lags"] > 0:
                if exog:
                    Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), np.ones((len(F_Z_t),1)),exog])
                else:
                    Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), np.ones((len(F_Z_t),1))])
            else: 
                Xm = np.ones((len(F_Z_t),1))
        else:
            if exog:
                Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), (1-F_Z_t), F_Z_t, exog])
            else:
                Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), (1-F_Z_t), F_Z_t])
    
    else: 
        if inter_regspec == 0 and trend_regspec == 0:
            if exog:
                Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), np.ones((len(F_Z_t),1)),np.array(range(len(F_Z_t))).reshape(-1,1), exog])
            else:
                Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), np.ones((len(F_Z_t),1)),np.array(range(len(F_Z_t))).reshape(-1,1)])
                
        elif inter_regspec == 1 and trend_regspec == 0: 
            if exog:
                Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), (1-F_Z_t), F_Z_t, np.array(range(len(F_Z_t))).reshape(-1,1), exog])
            else:
                Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), (1-F_Z_t), F_Z_t, np.array(range(len(F_Z_t))).reshape(-1,1)])
        
        elif inter_regspec == 0 and trend_regspec == 1: 
            if exog:
                Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), np.ones((len(F_Z_t),1)),(1-F_Z_t) * np.array(range(len(F_Z_t))).reshape(-1,1), F_Z_t * np.array(range(len(F_Z_t))).reshape(-1,1), exog])
            else:
                Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), np.ones((len(F_Z_t),1)),(1-F_Z_t) * np.array(range(len(F_Z_t))).reshape(-1,1), F_Z_t * np.array(range(len(F_Z_t))).reshape(-1,1)])
        
        elif inter_regspec == 1 and trend_regspec == 1: 
            if exog:
                Xm = np.block([lagged_vars * np.tile(1-F_Z_t,(1,cX)), lagged_vars * np.tile(F_Z_t,(1,cX)), (1-F_Z_t), F_Z_t, (1-F_Z_t) * np.array(range(len(F_Z_t))).reshape(-1,1), F_Z_t * np.array(range(len(F_Z_t))).reshape(-1,1), exog])
            else:
                Xm = np.block([lagged_vars.values * np.tile(1 - F_Z_t,(1,cX)), lagged_vars.values * np.tile(F_Z_t,(1,cX)), (1-F_Z_t), F_Z_t, (1-F_Z_t) * np.array(range(len(F_Z_t))).reshape(-1,1), F_Z_t * np.array(range(len(F_Z_t))).reshape(-1,1)])
    
    # Estimate VAR coefficients using GLS
    num0 = 0
    den0 = 0
    
    for t in range(0,T):
        wgt_m = inv(omega_1 * (1 - M_Z_t[t]) + omega_2 * M_Z_t[t])
        num0_mat = Xm[t:].T @ y[t:].values @ wgt_m
        
        num0 = num0 + num0_mat.flatten("F")
        den0 = den0 + np.kron(wgt_m, Xm[t:].T @ Xm[t:])
        
    betaK = inv(den0) @ num0
    betaK1 = unvec(betaK,cX*2+trendON*(1+trend_regspec)+1+inter_regspec+n_periods, nvar)
    
    # VAR coefficients in regime 0
    beta0 = betaK1[:cX,:].T
    # VAR coefficients in regime 1
    beta1 = betaK1[cX:2*cX,:].T
    
    # compute coefficients on the constant term and any other control variables
    beta_const = betaK1[2*cX:2*cX+1,:].T
    beta_other = betaK1[2*cX+2:,:]
    
    # compute residuals
    residsM = y.values - Xm @ betaK1
    
    # compute log-likelihood
    logL = 0
    
    for i in range(T):
        # compute time-varying parameters
        omega_t = omega_1 * (1 - M_Z_t[i]) + omega_2 * M_Z_t[i]
        
        e_t = residsM[i]
        
        logL = logL - 0.5*(np.log(det(omega_t)) + e_t@inv(omega_t)@e_t.T)
    
    return logL, beta0, beta1, beta_const, beta_other, omega_1,omega_2,theta,gamma,residsM

In [38]:
def max_LSTVAR_mleS_struct(X,state,lagged_vars,y,exog,T,params):
    
    omega_1, omega_2, theta = vec2matScov(X)
    
    if theta<0:
        if np.min(eig(omega_1)[0])>0 and np.min(eig(omega_2)[0])>0:
            gamma = theta
            n_periods = np.array(exog).shape[0]
            obj_logL, beta0, beta1, beta_const, beta_other  = LSTVAR_struct(theta,gamma,state,lagged_vars,y,exog,omega_1,omega_2,s_desc["T"],params["c_case"],params["lags"],params["num_vars"],n_periods)
            
            # restriction of the speed of change in the regime
            penalty_term0 = (theta - s_prior["theta0"])**2 * s_prior["scale_penalty"]
            
            if params["lags"]>0:
                penalty_term1 = 0.5 * sum(sum((beta1 - s_prior["prior_mean"])**2 / s_prior["prior_var"]))
                penalty_term2 = 0.5 * sum(sum((beta0 - s_prior["prior_mean"])**2 / s_prior["prior_var"]))
            else:
                penalty_term1 = 0
                penalty_term2 = 0
                
            # bayesian restriction: VAR covariance matrix of residuals
            m1 = np.tril(omega_1 - s_prior["omega_lin"])
            m2 = np.tril(omega_2 - s_prior["omega_lin"])
            
            penalty_term3 = 0.5 * np.sum(np.sum(m1[m1!=0].flatten("F")**2 / s_prior["omega_var"]))
            penalty_term4 = 0.5 * np.sum(np.sum(m2[m2!=0].flatten("F")**2 / s_prior["omega_var"]))
            
            obj_logL = -obj_logL + penalty_term0 + penalty_term1 + penalty_term2 + penalty_term3 + penalty_term4
            
        else:
             obj_logL = 1e12
    else:
        obj_logL = 1e12
    
    return obj_logL

In [39]:
def obj_logLoptimizeMCMC_struct(X,draws,params):
    print('Markov chain Monte Carlo...')
    # initial value and the fit
    J1 = max_LSTVAR_mleS_struct(X,state,lagged_vars,y,exog,T,params)
    
    # matrices to store draws and the fit
    A_mat = np.zeros((draws,len(X)))
    beta0_mat = np.zeros((draws,params["num_vars"]**2*params["lags"]))
    beta1_mat = np.zeros((draws,params["num_vars"]**2*params["lags"]))
    acceptrate = np.zeros((draws,1))
    valJ = np.zeros((draws,1))
    
    # size of the draws for the MCMC procedures
    sigmaMH = 0.0005*np.ones((X.shape))
    
    if s_prior["scale_penalty"] == 0:
        sigmaMH[-1] = 0.05
    else: 
        sigmaMH[-1] = 0
    
    scaleSIGMA=1
    n_periods = np.array(exog).shape[0]
    
    logL, beta0E, beta1E, beta_const, beta_other, omega_1E,omega_2E,theta,gamma,residsM = vec2matSF_struct(X,state,lagged_vars,y,exog,s_desc["T"],params["c_case"],params["lags"],params["num_vars"],n_periods)
    
    i = 1 
    np.random.seed(5318008) #set seed
    shocks0 = np.random.randn(X.shape[0],draws*5)
    counterB = 1
    counterX = 0
    bar = progressbar.ProgressBar(max_value=draws) # initialize progressbar
    
    while i < draws:
        A1 = X + sigmaMH * shocks0[:,counterB] # draw new stock to paramter value
        counterB = counterB + 1
        
        omega_1_candidate, omega_2_candidate, theta_candidate = vec2matScov(A1)
        
        # check if eigenvalues of Omega0 and Omega1 are greater than zero
        # if not, discard the candidate vector of parameters
        
        if np.min(eig(omega_1_candidate)[0])>0 and np.min(eig(omega_2_candidate)[0])>0:
            n_periods = np.array(exog).shape[0]
            J2 = max_LSTVAR_mleS_struct(A1,state,lagged_vars,y,exog,T,params)
        
        # stochastically accept/reject candidate values
        v0 = np.exp(J1-J2) # compute the exp of ration of loss functions
        v1 = min(v0,1) # determine accept probability
        U = np.random.randn(1,1) #draw rv from uniform to decide whether accept
        
        if U < v1: #accept if the U is less than acceptance rate
            X = A1
            acceptrate[i] = 1
            valJ[i] = J2
            J1 = J2
            logL, beta0E, beta1E, beta_const, beta_other, omega_1E,omega_2E,theta,gamma,residsM = vec2matSF_struct(X,state,lagged_vars,y,exog,s_desc["T"],params["c_case"],params["lags"],params["num_vars"],n_periods)
        else:
            acceptrate[i] = 0
            valJ[i] = J1
    
    
        A_mat[i] = X.T
        beta_const_mat = []
        beta_other_mat = []
        
        if params["lags"]>0:
            # adjust size of the shocks on the fly
            beta0_mat[i,:] = beta0E.flatten("F").T
            beta1_mat[i,:] = beta1E.flatten("F").T
            
            beta_other_mat.append(beta_other.flatten("F").T)
            beta_const_mat.append(beta_const.flatten("F").T)
        i+=1
        bar.update(i)
        counterX = 0
        
        # adjust size of the shocks on the fly
        if i < 400000:
            if i % 500 == 0:
                if np.mean(acceptrate[max(1,i-500):i]) > 0.35:
                    sigmaMH = 1.1 * sigmaMH
                    scaleSIGMA = 1.1 * scaleSIGMA
                elif np.mean(acceptrate[max(1,i-500):i]) > 0.25:
                    sigmaMH = 0.9 * sigmaMH
                    scaleSIGMA = 0.9 * scaleSIGMA
        else:
            counterX += 1
        
        if counterX > 200:#too many draws - Go back to previous successful draw
            X = A_mat[i-1,:].T
            J1 = valJ[i-1]
            counterX = 0
    
    return A_mat, beta0_mat,beta1_mat, acceptrate,valJ,beta_const_mat, beta_other_mat

In [40]:
draws = 100
burn_in = int(draws*0.2)

In [41]:
A_mat, beta0_mat,beta1_mat, acceptrate,valJ,beta_const_mat, beta_other_mat = obj_logLoptimizeMCMC_struct(param0_MCMC,draws,params)

Markov chain Monte Carlo...


100% (100 of 100) |######################| Elapsed Time: 0:00:12 ETA:  00:00:00

In [42]:
# clean results
A_mat = A_mat[burn_in:]
post_length = A_mat.shape[0]
beta0_mat = beta0_mat[burn_in:]
beta1_mat = beta1_mat[burn_in:]

In [43]:
omega_1,omega_2,theta = vec2matScov(np.mean(A_mat.T,axis=1))

In [44]:
# cholesky
s1 = cholesky(omega_1)
s2 = cholesky(omega_2)

In [45]:
beta0=unvec(np.mean(beta0_mat.T,axis=1),y.shape[1],y.shape[1]*params["lags"])
beta1=unvec(np.mean(beta1_mat.T,axis=1),y.shape[1],y.shape[1]*params["lags"])

# Identified impulse response

In [46]:
IRF1 = np.ones((params["h"],y.shape[1],post_length))*np.NaN
IRF2 = np.ones((params["h"],y.shape[1],post_length))*np.NaN

In [47]:
def dyn_multipliers(A,c_case, params):
    
    p = params["lags"]
    h = params["h"]
    n = y.shape[1]

    AR = A[0:].T

    AR_3d = np.empty((n,n,p))

    for lag in range(1,p+1):
        AR_3d[:,:,lag-1] = AR[(lag-1)*n:lag*n]


    C = np.empty((n,n,p+h))
    C[:,:,p-1] = 0
    C[:,:,p] = np.eye(n)

    for t in range(p+2,p+h):
        acc = 0
        for j in range(1,p+1):
            acc = acc + AR_3d[:,:,j-1]@C[:,:,t-j]
        C[:,:,t] = acc

    C = C[:,:,p:]
    return C

In [48]:
A1 = unvec(beta0_mat[1],y.shape[1],y.shape[1]*params["lags"])
C1 = dyn_multipliers(A1,c_case,params)

In [49]:
shock = np.zeros(n)
shock[params["shockposition"]-1] = 1 

for hh in range(h):
    IRF1[hh,:,i] = (C1[:,:,hh]*s1@shock).reshape(-1,1).T

NameError: name 'i' is not defined

In [ ]:
IRF1[hh,:,i]

In [50]:
for i in range(post_length):
    om_1,om_2,theta_IRF = vec2matScov(A_mat[i])
    
    s1 = cholesky(om_1)
    s2 = cholesky(om_2)
    c_case = 0 #parameters for constant not included in A&G
        
    # Impulse response in state 1
    A1 = unvec(beta0_mat[i],y.shape[1],y.shape[1]*params["lags"])
    C1 = dyn_multipliers(A1,c_case,params)
    
    shock = np.zeros(n)
    shock[params["shockposition"]-1] = 1 
    
    for hh in range(h):
        IRF1[hh,:,i] = (C1[:,:,hh]*s1@shock).reshape(-1,1).T
    
    # Impulse response in state 2
    
    A2 = unvec(beta1_mat[i].T,y.shape[1],y.shape[1]*params["lags"])
    C2 = dyn_multipliers(A2,c_case,params)
    
    shock = np.zeros(n)
    shock[params["shockposition"]-1] = 1 
    
    for hh in range(h):
        IRF2[hh,:,i] = (C2[:,:,hh]*s2@shock).reshape(-1,1).T

In [51]:
# Upper and lower thresholds
CI_LO = (100-params["CI"])/2
CI_UP = 100-params["CI"]

In [52]:
np.mean(IRF1,axis=2)

array([[0.00000000e+000, 0.00000000e+000, 4.09417725e-001],
       [0.00000000e+000,             nan, 1.25260763e+256],
       [            nan,             nan,             nan],
       [            nan,             nan,             nan],
       [            nan,             nan,             nan],
       [            nan,             nan,             nan],
       [            nan,             nan,             nan],
       [            nan,             nan,             nan],
       [            nan,             nan,             nan],
       [            nan,             nan,             nan],
       [            nan,             nan,             nan],
       [            nan,             nan,             nan],
       [            nan,             nan,             nan],
       [            nan,             nan,             nan],
       [            nan,             nan,             nan],
       [            nan,             nan,             nan],
       [            nan,             nan

In [ ]:
pd.read: